# Get the necessary packages

In [ ]:
!git clone https://github.com/asellam/sitrex.git
%cd sitrex

Cloning into 'sitrex'...
remote: Enumerating objects: 146, done.
remote: Counting objects: 100% (146/146), done.
remote: Compressing objects: 100% (100/100), done.
remote: Total 146 (delta 77), reused 99 (delta 45), pack-reused 0 (from 0)
Receiving objects: 100% (146/146), 1.78 MiB | 10.19 MiB/s, done.
Resolving deltas: 100% (77/77), done.
/content/sitrex


In [ ]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opencv-contrib-python to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 95.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 10.0 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: opencv-contrib-python
    Found existing installation: opencv-contrib-python 4.12.0.88
    Uninstalling openc

In [ ]:
# After session restart, continue running from this cell not from the beginning
%cd sitrex

/content/sitrex


# Download the dataset

In [ ]:
import kagglehub

path = kagglehub.dataset_download('abdellah213/sitrex-dataset')

100%|██████████| 418M/418M [00:03<00:00, 127MB/s] 

Extracting files...


# Imports and Setup

In [ ]:
import sys
sys.path.append('.')
# Import necessary libraries
import os
from sklearn.model_selection import train_test_split
from sitrex.preprocessing import load_dataset, preprocess_data, SimilarityDataset, UsefulnessDataset
from sitrex.model import angle_usefulness_model, angle_similarity_model, TQDMProgressBar
import tensorflow as tf

import numpy as np
# Define constants
DATASET_PATH = os.path.join(path, 'dataset')
# EarlyStop save path for the Siamese Transformer
MODEL_SAVE_PATH_SIMILARITY = './angle_similarity_model.weights.h5'
# EarlyStop save path for the Angle Usefulness Transformer
MODEL_SAVE_PATH_USEFULNESS = './angle_usefulness_model.weights.h5'
# Maximum sequence length (L)
MAXLEN = 100
# Batch Size
batch_size = 32

# Loading the data

<u>**angle_type:**</u>

*   **unsigned:** Unsigned joint angles calculated using arccos
*   **signed:** Signed joint angles calculated using arctan2 without projection on any anatomical plane
*   **max-energy:** Signed joint angles calculated using arctan2 after projection the anatomical plane that introduces maximum variance in angle values over the temporal sequence
*   **sagittal:** Signed joint angles calculated using arctan2 after projection the sagittal anatomical plane
*   **coronal:** Signed joint angles calculated using arctan2 after projection the coronal (frontal) anatomical plane
*   **transverse:** Signed joint angles calculated using arctan2 after projection the transverse anatomical plane
*   **all:** Three signed angles per joint calculated using arctan2 after projection all three anatomical plane

In [ ]:
all_sequences, labels, exercise_angles = load_dataset(DATASET_PATH, angle_type='sagittal')

Loading dataset: 100%|██████████| 727/727 [11:45<00:00,  1.03it/s]

Loaded 727 sequences with labels.


# Preprocessing

In [ ]:
processed_sequences, numerical_labels, label_angles = preprocess_data(all_sequences, labels, exercise_angles)

# Tensorflow Dataset and Model Routines

# K-Fold Cross-Validation

In [ ]:
def split_by_fold(data_x, data_y, folds, test):
    train_x, test_x, train_y, test_y = [], [], [], []
    for fold, split in enumerate(folds):
        for sample in split:
            if fold == test:
                test_x.append(data_x[sample])
                test_y.append(data_y[sample])
            else:
                train_x.append(data_x[sample])
                train_y.append(data_y[sample])
    return train_x, test_x, train_y, test_y

In [ ]:
# Preparing the 5 folds
k = 5
N = len(numerical_labels)
indexes = np.arange(N)
np.random.seed(42)
np.random.shuffle(indexes)

fold_size = int(np.ceil(N / k))

folds = []
for i in range(k):
    start = i * fold_size
    end = min(start + fold_size, N)
    folds.append(indexes[start:end])

print(folds)

[array([513, 693, 377,  33,  63, 467, 346, 511, 148, 388, 174,  65, 469,
       428, 707, 350, 210,  72, 449,  78,  54,  39,  97, 211,  81, 557,
       629, 443, 514,  10, 361, 319, 676, 673, 231, 363, 158, 462, 296,
       578, 723, 396,  86, 329, 597, 235, 351, 568, 299, 685, 165, 571,
       164, 223, 539, 399, 281, 227, 714, 620, 199, 705, 155,  49, 332,
       101, 477, 234, 437, 196,  23, 266,  77, 212, 198, 109, 713, 327,
       528, 336, 209,  30, 602, 674, 480, 554, 393, 547, 118, 609, 133,
       688,  84,  79, 213, 433, 448, 181,  31, 254, 598, 215, 412, 591,
       275,  55, 679, 594, 691, 300,  76,   2, 506, 464, 192,  60, 722,
       120, 362, 429, 314, 218, 220, 634, 260, 405, 244, 426, 431,  90,
       331, 538,  69, 204, 131,  44,  70, 349, 717, 569, 135, 601, 239,
       579, 642, 432]), array([136, 286,   6, 657, 587, 334, 250, 145, 338, 132, 306, 660,  41,
       108, 292,  56, 417, 690, 333, 537,  24, 404, 465, 666, 311, 457,
       110,  82,  51, 394, 582, 497, 29

In [ ]:
epochs = 80
for seq_module in ['gru', 'lstm', 'transformer']:
    print(f'Performing k-fold cross-validation for the Angle Usefulness {seq_module} ...')
    for test in range(k):
        print(f'Testing on Fold #{test+1}/{k} ...')
        X_train, X_test, y_train, y_test = split_by_fold(
            processed_sequences,
            numerical_labels,
            folds,
            test
        )

        X_train, X_val, y_train, y_val = train_test_split(
            X_train,
            y_train,
            test_size=0.2,
            random_state=42
        )

        y_train = np.array(y_train, dtype=np.int32)
        y_test = np.array(y_test, dtype=np.int32)
        y_val = np.array(y_val, dtype=np.int32)

        Y_train = np.empty((len(X_train), len(X_train[0][0])), np.float32)
        Y_val = np.empty((len(X_val), len(X_train[0][0])), np.float32)
        Y_test = np.empty((len(X_test), len(X_train[0][0])), np.float32)

        for (Y, y) in [(Y_train, y_train), (Y_val, y_val), (Y_test, y_test)]:
            for i in range(Y.shape[0]):
                for angle in range(Y.shape[1]):
                    if angle in label_angles[y[i]]:
                        Y[i, angle] = 1
                    else:
                        Y[i, angle] = 0

        usefulness_model = angle_usefulness_model(num_angles=len(X_train[0][0]), maxlen=MAXLEN, module=seq_module, lr=1e-3)

        pbar_callback = TQDMProgressBar(epochs=epochs)

        callback = tf.keras.callbacks.ModelCheckpoint(
            MODEL_SAVE_PATH_USEFULNESS,
            monitor="val_binary_accuracy",
            verbose=0,
            save_best_only=True,
            save_weights_only=True,
            mode="max",
            save_freq="epoch",
            initial_value_threshold=None,
        )

        train_ds = UsefulnessDataset(X_train, Y_train, y_train, batch_size=batch_size, maxlen=MAXLEN, train=True)
        val_ds = UsefulnessDataset(X_val, Y_val, y_val, batch_size=min(batch_size, len(X_val)), maxlen=MAXLEN, train=False)

        history = usefulness_model.fit(
            train_ds,
            validation_data=val_ds,
            batch_size=batch_size,
            epochs=epochs,
            callbacks=[callback, pbar_callback],
            verbose=0, # Change this to 1 if you want progressive display while training
        )

        # Load the trained classifier model
        usefulness_model.load_weights(
            MODEL_SAVE_PATH_USEFULNESS
        )

        test_ds = UsefulnessDataset(X_test, Y_test, y_test, batch_size=batch_size, maxlen=MAXLEN, train=False)
        usefulness_model.evaluate(test_ds, verbose=1)

Performing k-fold cross-validation for the Angle Usefulness gru ...
Testing on Fold #1/5 ...


Training Progress: 100%|██████████| 80/80 [01:11<00:00,  1.13epoch/s, Precision=0.8370, Recall=0.6948, binary_accuracy=0.8962, loss=0.2481, val_Precision=0.7201, val_Recall=0.5434, val_binary_accuracy=0.8225, val_loss=0.3770]

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - Precision: 0.7561 - Recall: 0.6143 - binary_accuracy: 0.8510 - loss: 0.3460


Testing on Fold #2/5 ...


Training Progress: 100%|██████████| 80/80 [01:10<00:00,  1.14epoch/s, Precision=0.8194, Recall=0.6861, binary_accuracy=0.8906, loss=0.2587, val_Precision=0.7785, val_Recall=0.5939, val_binary_accuracy=0.8474, val_loss=0.3459]

...

1/4 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - Precision: 0.8128 - Recall: 0.8042 - binary_accuracy: 0.9022 - loss: 0.2279

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - Precision: 0.8224 - Recall: 0.7639 - binary_accuracy: 0.8966 - loss: 0.2467
Testing on Fold #5/5 ...


Training Progress: 100%|██████████| 80/80 [01:04<00:00,  1.24epoch/s, Precision=0.8905, Recall=0.8268, binary_accuracy=0.9352, loss=0.1633, val_Precision=0.8589, val_Recall=0.7381, val_binary_accuracy=0.9013, val_loss=0.2540]

1/4 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - Precision: 0.7101 - Recall: 0.6154 - binary_accuracy: 0.8315 - loss: 0.3780

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - Precision: 0.7585 - Recall: 0.6543 - binary_accuracy: 0.8553 - loss: 0.3410


In [ ]:
epochs = 60
for seq_module in ['gru', 'lstm', 'transformer']:
    print(f'Performing k-fold cross-validation for the Siamese {seq_module} ...')
    for test in range(k):
        print(f'Testing on Fold #{test+1}/{k} ...')
        X_train, X_test, y_train, y_test = split_by_fold(
            processed_sequences,
            numerical_labels,
            folds,
            test
        )

        X_train, X_val, y_train, y_val = train_test_split(
            X_train,
            y_train,
            test_size=0.2,
            random_state=42
        )

        y_train = np.array(y_train, dtype=np.int32)
        y_test = np.array(y_test, dtype=np.int32)
        y_val = np.array(y_val, dtype=np.int32)

        train_batches = 10 * len(X_train) // batch_size
        val_batches = 10 * len(X_val) // batch_size
        train_ds = SimilarityDataset(X_train, y_train, num_batches=train_batches, batch_size=batch_size, label_angles=label_angles, maxlen=MAXLEN, train=True)
        val_ds = SimilarityDataset(X_val, y_val, num_batches=val_batches, batch_size=batch_size, label_angles=label_angles, maxlen=MAXLEN, train=False)

        callback = tf.keras.callbacks.ModelCheckpoint(
            MODEL_SAVE_PATH_SIMILARITY,
            monitor="val_binary_accuracy",
            verbose=0,
            save_best_only=True,
            save_weights_only=True,
            mode="max",
            save_freq="epoch",
            initial_value_threshold=None,
        )

        pbar_callback = TQDMProgressBar(epochs=epochs)

        # Build and train Siamese model
        similarity_model = angle_similarity_model(num_angles=len(X_train[0][0]), maxlen=MAXLEN, module=seq_module, lr=1e-3)
        history = similarity_model.fit(
            train_ds,
            validation_data=val_ds,
            epochs=epochs,
            callbacks=[callback, pbar_callback],
            verbose=0,  # Change this to 1 if you want progressive display while training
        )

        similarity_model.load_weights(
            MODEL_SAVE_PATH_SIMILARITY,
        )

        # Make a large dataset in terms of pair in order to have a more reliable test
        # since we can't test on all possible pairs (very huge number)
        test_batches = 1000 * len(X_test) // batch_size
        # Evaluate the Siamese model on the test set
        test_ds = SimilarityDataset(X_test, y_test, num_batches=test_batches, batch_size=batch_size, label_angles=label_angles, maxlen=MAXLEN, train=False)
        similarity_model.evaluate(test_ds, verbose=1)

Performing k-fold cross-validation for the Siamese gru ...
Testing on Fold #1/5 ...


Training Progress: 100%|██████████| 60/60 [06:57<00:00,  6.97s/epoch, Precision=0.8665, Recall=0.7572, binary_accuracy=0.9159, loss=0.2037, val_Precision=0.7912, val_Recall=0.6530, val_binary_accuracy=0.8732, val_loss=0.3136]

   1/4562 ━━━━━━━━━━━━━━━━━━━━ 3:50 51ms/step - Precision: 0.8025 - Recall: 0.7065 - binary_accuracy: 0.8832 - loss: 0.3148

4562/4562 ━━━━━━━━━━━━━━━━━━━━ 151s 33ms/step - Precision: 0.7955 - Recall: 0.6748 - binary_accuracy: 0.8803 - loss: 0.3130
Testing on Fold #2/5 ...


Training Progress: 100%|██████████| 60/60 [06:54<00:00,  6.90s/epoch, Precision=0.8631, Recall=0.7600, binary_accuracy=0.9151, loss=0.2022, val_Precision=0.7870, val_Recall=0.7032, val_binary_accuracy=0.8824, val_loss=0.2890]

...

   1/4562 ━━━━━━━━━━━━━━━━━━━━ 4:18 57ms/step - Precision: 0.8917 - Recall: 0.7292 - binary_accuracy: 0.9062 - loss: 0.2086

4562/4562 ━━━━━━━━━━━━━━━━━━━━ 178s 39ms/step - Precision: 0.6529 - Recall: 0.5737 - binary_accuracy: 0.8276 - loss: 0.5436
Testing on Fold #3/5 ...


Training Progress: 100%|██████████| 60/60 [07:14<00:00,  7.24s/epoch, Precision=0.8708, Recall=0.7732, binary_accuracy=0.9198, loss=0.1887, val_Precision=0.7974, val_Recall=0.7193, val_binary_accuracy=0.8876, val_loss=0.2956]


4562/4562 ━━━━━━━━━━━━━━━━━━━━ 164s 36ms/step - Precision: 0.7692 - Recall: 0.6986 - binary_accuracy: 0.8791 - loss: 0.3973
Testing on Fold #4/5 ...


Training Progress:  85%|████████▌ | 51/60 [05:52<01:01,  6.87s/epoch, Precision=0.8679, Recall=0.7702, binary_accuracy=0.9183, loss=0.1996, val_Precision=0.7665, val_Recall=0.6535, val_binary_accuracy=0.8676, val_loss=0.3379]

# One-shot Generalization Evaluation

In [ ]:
tests = [[3, 4, 13, 14, 15, 17], [1, 3, 4, 13, 14, 20], [1, 3, 6, 13, 19, 20], [1, 3, 7, 13, 14, 16], [1, 3, 5, 13, 14, 19], [1, 2, 3, 13, 19, 21], [0, 3, 6, 7, 13, 19], [1, 3, 4, 13, 14, 21], [3, 7, 13, 14, 16, 17], [1, 3, 10, 13, 20, 21]]

In [ ]:
# Function to split data by specifying test labels
def split_by_labels(data_x, data_y, test_labels):
    X_train, X_test, y_train, y_test = [], [], [], []
    for sample_x, sample_y in zip(data_x, data_y):
        if sample_y in test_labels:
            X_test.append(sample_x)
            y_test.append(sample_y)
        else:
            X_train.append(sample_x)
            y_train.append(sample_y)
    return X_train, X_test, y_train, y_test

In [ ]:
epochs = 80
print('Performing one-shot validation for the Angle Usefulness Transformer ...')
for test in tests:
    print(f'Testing on Exercises: {test} ...')
    X_train, X_test, y_train, y_test = split_by_labels(
        processed_sequences,
        numerical_labels,
        test_labels=test,
    )

    X_train, X_val, y_train, y_val = train_test_split(
        X_train,
        y_train,
        test_size=0.2,
        random_state=42
    )

    y_train = np.array(y_train, dtype=np.int32)
    y_test = np.array(y_test, dtype=np.int32)
    y_val = np.array(y_val, dtype=np.int32)

    Y_train = np.empty((len(X_train), len(X_train[0][0])), np.float32)
    Y_val = np.empty((len(X_val), len(X_train[0][0])), np.float32)
    Y_test = np.empty((len(X_test), len(X_train[0][0])), np.float32)

    for (Y, y) in [(Y_train, y_train), (Y_val, y_val), (Y_test, y_test)]:
        for i in range(Y.shape[0]):
            for angle in range(Y.shape[1]):
                if angle in label_angles[y[i]]:
                    Y[i, angle] = 1
                else:
                    Y[i, angle] = 0

    usefulness_model = angle_usefulness_model(maxlen=MAXLEN, module='transformer', lr=1e-4)

    callback = tf.keras.callbacks.ModelCheckpoint(
        MODEL_SAVE_PATH_USEFULNESS,
        monitor="val_binary_accuracy",
        verbose=0,
        save_best_only=True,
        save_weights_only=True,
        mode="max",
        save_freq="epoch",
        initial_value_threshold=None,
    )

    pbar_callback = TQDMProgressBar(epochs=epochs)

    train_ds = UsefulnessDataset(X_train, Y_train, y_train, batch_size=batch_size, maxlen=MAXLEN, train=True)
    val_ds = UsefulnessDataset(X_val, Y_val, y_val, batch_size=min(batch_size, len(X_val)), maxlen=MAXLEN, train=False)

    history = usefulness_model.fit(
        train_ds,
        validation_data=val_ds,
        batch_size=batch_size,
        epochs=epochs,
        callbacks=[callback, pbar_callback],
        verbose=0,  # Change this to 1 if you want progressive display while training
    )

    # Load the trained classifier model
    usefulness_model.load_weights(
        MODEL_SAVE_PATH_USEFULNESS
    )

    test_ds = UsefulnessDataset(X_test, Y_test, y_test, batch_size=batch_size, maxlen=MAXLEN, train=False)
    usefulness_model.evaluate(test_ds, verbose=1)

In [ ]:
epochs = 60
print('Performing one-shot validation for the Siamese Transformer ...')
for test in tests:
    print(f'Testing on Exercises: {test} ...')

    X_train, X_test, y_train, y_test = split_by_labels(
        processed_sequences,
        numerical_labels,
        test_labels=test,
    )

    X_train, X_val, y_train, y_val = train_test_split(
        X_train,
        y_train,
        test_size=0.2,
        random_state=42
    )
    train_batches = 10 * len(X_train) // batch_size
    val_batches = 10 * len(X_val) // batch_size
    train_ds = SimilarityDataset(X_train, y_train, num_batches=train_batches, batch_size=batch_size, label_angles=label_angles, maxlen=MAXLEN, train=True)
    val_ds = SimilarityDataset(X_val, y_val, num_batches=val_batches, batch_size=batch_size, label_angles=label_angles, maxlen=MAXLEN, train=False)

    callback = tf.keras.callbacks.ModelCheckpoint(
        MODEL_SAVE_PATH_SIMILARITY,
        monitor="val_binary_accuracy",
        verbose=0,
        save_best_only=True,
        save_weights_only=True,
        mode="max",
        save_freq="epoch",
        initial_value_threshold=None,
    )

    pbar_callback = TQDMProgressBar(epochs=epochs)

    # Build and train Siamese model
    similarity_model = angle_similarity_model(maxlen=MAXLEN, module='transformer', lr=1e-3)
    history = similarity_model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=epochs,
        callbacks=[callback, pbar_callback],
        verbose=0,  # Change this to 1 if you want progressive display while training
    )

    # Load the trained Siamese model
    similarity_model.load_weights(
        MODEL_SAVE_PATH_SIMILARITY,
    )

    # Make a large dataset in terms of pair in order to have a more reliable test
    # since we can't test on all possible pairs (very huge number)
    test_batches = 1000 * len(X_test) // batch_size
    # Evaluate the Siamese model on the test set
    test_ds = SimilarityDataset(X_test, y_test, num_batches=test_batches, batch_size=batch_size, label_angles=label_angles, maxlen=MAXLEN, train=False)
    similarity_model.evaluate(test_ds, verbose=1)